# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [56]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import re

In [57]:
def retrieve_titles(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    titles = book_container.findAll('h3')
    final_titles = [h3.find('a').attrs['title'] for h3 in book_container.findAll('h3')]
    return final_titles
    #Your code here

In [58]:
html_page = requests.get('http://books.toscrape.com/')
soup = BeautifulSoup(html_page.content, 'html.parser') 

In [59]:
retrieve_titles(soup)

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [60]:
def retrieve_ratings(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    regex = re.compile("star-rating (.*)")
    book_container.findAll('p', {"class" : regex})
    star_ratings = []
    for p in book_container.findAll('p', {"class" : regex}):
        star_ratings.append(p.attrs['class'][-1])
    star_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    star_ratings = [star_dict[s] for s in star_ratings]
    return star_ratings
    #Your code here


In [61]:
retrieve_ratings(soup)

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [62]:
def retrieve_prices(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    book_container.findAll('p', class_="price_color")
    prices = [p.text for p in book_container.findAll('p', class_="price_color")]
    formatted_prices = [float(p[1:]) for p in prices]
    return formatted_prices
    #Your code here

In [63]:
retrieve_prices(soup)

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [64]:
def retrieve_availabilities(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    avail = [a.text.strip() for a in book_container.findAll('p', class_="instock availability")]
    return avail
    
    #Your code here

In [65]:
retrieve_availabilities(soup)

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [70]:
#Your code here
final_titles = []
star_ratings = []
formatted_prices = []
avail = []

for i in range(1, 51):
    if i == 1: # sets case for homepage
        url = 'http://books.toscrape.com/'
    else:
        url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
    final_titles += retrieve_titles(soup)
    star_ratings += retrieve_ratings(soup)
    formatted_prices += retrieve_prices(soup)
    avail += retrieve_availabilities(soup)
    
df = pd.DataFrame([final_titles, star_ratings, formatted_prices, avail]).transpose()
df.columns = ['Title', 'Star_Rating', 'Price (GBP)', 'Availability']
print(len(df))
df[:25]

1000


,Title,Star_Rating,Price (GBP),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!